In [2]:
# Build the dynamic WHERE clause for NULL checks
list_of_columns = ['bic_payable_vendor_nbr','vendor_number','vendor_name']
process_status = "'U','Z'"
request_id = 7
null_conditions = " OR ".join([f"{col} IS NULL" for col in list_of_columns])

# Construct the SQL query
get_all_query = f"""
    SELECT stg_seq_id, PROCESS_STATUS, ERROR_MESSAGE, {', '.join(list_of_columns)}
    FROM XXPO_ORCL2CSPOMS_VEND_STG 
    WHERE PROCESS_STATUS IN ({process_status})
    AND request_id = {request_id}
    AND ({null_conditions})
"""

print(get_all_query)



    SELECT stg_seq_id, PROCESS_STATUS, ERROR_MESSAGE, bic_payable_vendor_nbr, vendor_number, vendor_name
    FROM XXPO_ORCL2CSPOMS_VEND_STG 
    WHERE PROCESS_STATUS IN ('U','Z')
    AND request_id = 7
    AND (bic_payable_vendor_nbr IS NULL OR vendor_number IS NULL OR vendor_name IS NULL)



In [ ]:
stg_seq_id	process_status	error_message	bic_payable_vendor_nbr	vendor_nbr	vendor_name
309269	Z		32777		

In [ ]:
# I am using postgres and i have a postgres cursor post_cur

def check_and_update_null_columns(post_cur, post_conn, process_status, list_of_columns, request_id):
    try:
        # Build the dynamic WHERE clause for NULL checks
        null_conditions = " OR ".join([f"{col} IS NULL" for col in list_of_columns])
        
        # Construct the SQL query to fetch the data
        get_all_query = f"""
            SELECT stg_seq_id, PROCESS_STATUS, ERROR_MESSAGE, {', '.join(list_of_columns)}
            FROM XXPO_ORCL2CSPOMS_VEND_STG 
            WHERE PROCESS_STATUS IN ({process_status})
            AND request_id = {request_id}
            AND ({null_conditions})
        """
        
        # Execute the query and fetch the DataFrame
        post_cur.execute(get_all_query)
        df = pd.DataFrame(post_cur.fetchall(), columns=[desc[0] for desc in post_cur.description])
        
        if df.empty:
            print("No rows to have missing columns to Check.")
            return
    '''So now all the df I have queried will have one or more null columns in the list_of_columns 
      so I need to check for those and make the same df to error df update process_status to E 
      and error_message column in error_df should become like check in that row which all columns are null update 'Columns missing - {the columns here}' '''
        print(error_df)
        
        # Commit the transaction
        post_conn.commit()
        
        print("Updates complete.")
    
    except Exception as e:
        print(f"Error checking and updating null columns:")
        print(f"Error: {e}")
        post_conn.rollback()

# Example usage
# Ensure you have a valid PostgreSQL connection
# pgsql_hook = PostgresHook(postgres_conn_id='your_postgres_conn_id')
# post_conn = pgsql_hook.get_conn()
# post_cur = post_conn.cursor()

list_of_columns = ['bic_payable_vendor_nbr', 'vendor_number', 'vendor_name']
process_status = "'U','Z'"
request_id = 7

check_and_update_null_columns(post_cur, post_conn, process_status, list_of_columns, request_id)


In [4]:
def generate_update_query(table_name, list_to_check):
    # Construct the parts of the CASE statement for error_message
    error_message_parts = []
    for column in list_to_check:
        condition = f"COALESCE(NULLIF({column} IS NULL, FALSE) * '{column} ', '')"
        error_message_parts.append(condition)

    # Join the error message parts
    error_message_expression = "CONCAT('Missing: ', " + ", ".join(error_message_parts) + ")"

    # Construct the SQL query
    query = f"""
    UPDATE {table_name}
    SET error_message = 
        CASE 
            WHEN {" OR ".join([f"{column} IS NULL" for column in list_to_check])} THEN
                {error_message_expression}
            ELSE 
                NULL
        END,
        process_status = 
        CASE 
            WHEN {" OR ".join([f"{column} IS NULL" for column in list_to_check])} THEN 'Error'
            ELSE 'Processed'
        END
    WHERE 
        {" OR ".join([f"{column} IS NULL" for column in list_to_check])};
    """

    return query

# Example usage:
list_to_check = ['BIC_BUYER_NUMBER', 'BIC_STATUS', 'BIC_PAYABLE_VENDOR_NBR', 'BIC_VENDOR_TYPE']


query = generate_update_query('XXPO_ORCL2CSPOMS_VEND_STG', list_to_check)
print(query)


    UPDATE XXPO_ORCL2CSPOMS_VEND_STG
    SET error_message = 
        CASE 
            WHEN BIC_BUYER_NUMBER IS NULL OR BIC_STATUS IS NULL OR BIC_PAYABLE_VENDOR_NBR IS NULL OR BIC_VENDOR_TYPE IS NULL THEN
                CONCAT('Missing: ', COALESCE(NULLIF(BIC_BUYER_NUMBER IS NULL, FALSE) * 'BIC_BUYER_NUMBER ', ''), COALESCE(NULLIF(BIC_STATUS IS NULL, FALSE) * 'BIC_STATUS ', ''), COALESCE(NULLIF(BIC_PAYABLE_VENDOR_NBR IS NULL, FALSE) * 'BIC_PAYABLE_VENDOR_NBR ', ''), COALESCE(NULLIF(BIC_VENDOR_TYPE IS NULL, FALSE) * 'BIC_VENDOR_TYPE ', ''))
            ELSE 
                NULL
        END,
        process_status = 
        CASE 
            WHEN BIC_BUYER_NUMBER IS NULL OR BIC_STATUS IS NULL OR BIC_PAYABLE_VENDOR_NBR IS NULL OR BIC_VENDOR_TYPE IS NULL THEN 'Error'
            ELSE 'Processed'
        END
    WHERE 
        BIC_BUYER_NUMBER IS NULL OR BIC_STATUS IS NULL OR BIC_PAYABLE_VENDOR_NBR IS NULL OR BIC_VENDOR_TYPE IS NULL;
    


In [5]:
# List of mandatory column names
mandatory_columns_list = [
    'BIC_BUYER_NUMBER',
    'BIC_STATUS',
    'BIC_PAYABLE_VENDOR_NBR',
    'BIC_VENDOR_TYPE'
]

# Generate the CASE statements for each column
missing_columns_case = ", ".join(
    [f"CASE WHEN {col} IS NULL THEN '{col}' ELSE NULL END" for col in mandatory_columns_list]
)

# Construct the final SQL query
query = f"""
UPDATE XXPO_ORCL2CSPOMS_VEND_STG
SET 
    error_message = CASE 
        WHEN {" OR ".join([f"{col} IS NULL" for col in mandatory_columns_list])} THEN
            CONCAT(
                'Missing - ',
                CONCAT_WS(', ',
                    {missing_columns_case}
                )
            )
        ELSE NULL
    END,
    process_status = CASE 
        WHEN {" OR ".join([f"{col} IS NULL" for col in mandatory_columns_list])} THEN 'E'
        ELSE 'P'
    END
WHERE 
    {" OR ".join([f"{col} IS NULL" for col in mandatory_columns_list])};
"""

print(query)



UPDATE XXPO_ORCL2CSPOMS_VEND_STG
SET 
    error_message = CASE 
        WHEN BIC_BUYER_NUMBER IS NULL OR BIC_STATUS IS NULL OR BIC_PAYABLE_VENDOR_NBR IS NULL OR BIC_VENDOR_TYPE IS NULL THEN
            CONCAT(
                'Missing - ',
                CONCAT_WS(', ',
                    CASE WHEN BIC_BUYER_NUMBER IS NULL THEN 'BIC_BUYER_NUMBER' ELSE NULL END, CASE WHEN BIC_STATUS IS NULL THEN 'BIC_STATUS' ELSE NULL END, CASE WHEN BIC_PAYABLE_VENDOR_NBR IS NULL THEN 'BIC_PAYABLE_VENDOR_NBR' ELSE NULL END, CASE WHEN BIC_VENDOR_TYPE IS NULL THEN 'BIC_VENDOR_TYPE' ELSE NULL END
                )
            )
        ELSE NULL
    END,
    process_status = CASE 
        WHEN BIC_BUYER_NUMBER IS NULL OR BIC_STATUS IS NULL OR BIC_PAYABLE_VENDOR_NBR IS NULL OR BIC_VENDOR_TYPE IS NULL THEN 'E'
        ELSE 'P'
    END
WHERE 
    BIC_BUYER_NUMBER IS NULL OR BIC_STATUS IS NULL OR BIC_PAYABLE_VENDOR_NBR IS NULL OR BIC_VENDOR_TYPE IS NULL;



In [6]:
# List of mandatory column names
mandatory_columns_list = [
    'BIC_BUYER_NUMBER',
    'BIC_STATUS',
    'BIC_PAYABLE_VENDOR_NBR',
    'BIC_VENDOR_TYPE'
]

# Generate the CASE statements for each column
missing_columns_case = ", ".join(
    [f"CASE WHEN COALESCE({col}, '') = '' THEN '{col}' ELSE NULL END" for col in mandatory_columns_list]
)

# Construct the final SQL query
query = f"""
UPDATE XXPO_ORCL2CSPOMS_VEND_STG
SET 
    error_message = CASE 
        WHEN {" OR ".join([f"COALESCE({col}, '') = ''" for col in mandatory_columns_list])} THEN
            CONCAT(
                'Missing - ',
                CONCAT_WS(', ',
                    {missing_columns_case}
                )
            )
        ELSE NULL
    END,
    process_status = CASE 
        WHEN {" OR ".join([f"COALESCE({col}, '') = ''" for col in mandatory_columns_list])} THEN 'E'
        ELSE 'P'
    END
WHERE 
    {" OR ".join([f"COALESCE({col}, '') = ''" for col in mandatory_columns_list])};
"""

print(query)



UPDATE XXPO_ORCL2CSPOMS_VEND_STG
SET 
    error_message = CASE 
        WHEN COALESCE(BIC_BUYER_NUMBER, '') = '' OR COALESCE(BIC_STATUS, '') = '' OR COALESCE(BIC_PAYABLE_VENDOR_NBR, '') = '' OR COALESCE(BIC_VENDOR_TYPE, '') = '' THEN
            CONCAT(
                'Missing - ',
                CONCAT_WS(', ',
                    CASE WHEN COALESCE(BIC_BUYER_NUMBER, '') = '' THEN 'BIC_BUYER_NUMBER' ELSE NULL END, CASE WHEN COALESCE(BIC_STATUS, '') = '' THEN 'BIC_STATUS' ELSE NULL END, CASE WHEN COALESCE(BIC_PAYABLE_VENDOR_NBR, '') = '' THEN 'BIC_PAYABLE_VENDOR_NBR' ELSE NULL END, CASE WHEN COALESCE(BIC_VENDOR_TYPE, '') = '' THEN 'BIC_VENDOR_TYPE' ELSE NULL END
                )
            )
        ELSE NULL
    END,
    process_status = CASE 
        WHEN COALESCE(BIC_BUYER_NUMBER, '') = '' OR COALESCE(BIC_STATUS, '') = '' OR COALESCE(BIC_PAYABLE_VENDOR_NBR, '') = '' OR COALESCE(BIC_VENDOR_TYPE, '') = '' THEN 'E'
        ELSE 'P'
    END
WHERE 
    COALESCE(BIC_BUYER_NUMBER, '')